In [ ]:
from pydantic import BaseModel
from typing import List, Dict, Optional
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import json
import os

# 定义文件路径
MOVIES_FILE = "./movie_data.json"
THEATERS_FILE = "./theater_data.json"
SHOWTIMES_FILE = "./showtime_data.json"
TICKETS_FILE = "./ticket_data.json"

# 定义电影模型
class Movie(BaseModel):
    movie_id: str
    title: str
    duration: int  # 单位：分钟
    genre: List[str]  # 电影类型
    language: str
    rating: float  # 评分，1-10

# 定义影厅模型
class Theater(BaseModel):
    theater_id: str
    name: str
    capacity: int
    type: List[str]  # 2D, 3D, IMAX等
    seating_layout: Dict[str, bool]  # 座位布局，True表示可用，False表示已预订

# 定义场次模型
class Showtime(BaseModel):
    showtime_id: str
    movie_id: str
    theater_id: str
    start_time: str
    end_time: str
    price: float
    available_seats: int
    booked_seats: List[str]  # 已预订的座位列表

# 定义电影票模型
class Ticket(BaseModel):
    ticket_id: str
    showtime_id: str
    movie_id: str
    theater_id: str
    seat_number: str
    user_name: str
    price: float
    booking_time: str

# 初始化电影数据
def initialize_movies_data():
    """初始化电影数据并写入JSON文件"""
    movies = {
        "M001": Movie(
            movie_id="M001",
            title="星际穿越",
            duration=169,
            genre=["科幻", "冒险", "剧情"],
            language="英语",
            rating=9.3
        ),
        "M002": Movie(
            movie_id="M002",
            title="泰坦尼克号",
            duration=194,
            genre=["爱情", "灾难", "剧情"],
            language="英语",
            rating=9.4
        ),
        "M003": Movie(
            movie_id="M003",
            title="盗梦空间",
            duration=148,
            genre=["科幻", "悬疑", "冒险"],
            language="英语",
            rating=9.1
        ),
        "M004": Movie(
            movie_id="M004",
            title="肖申克的救赎",
            duration=142,
            genre=["剧情", "犯罪"],
            language="英语",
            rating=9.7
        ),
        "M005": Movie(
            movie_id="M005",
            title="楚门的世界",
            duration=103,
            genre=["剧情", "科幻"],
            language="英语",
            rating=9.2
        ),
        "M006": Movie(
            movie_id="M006",
            title="千与千寻",
            duration=125,
            genre=["动画", "奇幻", "冒险"],
            language="日语",
            rating=9.3
        ),
        "M007": Movie(
            movie_id="M007",
            title="这个杀手不太冷",
            duration=110,
            genre=["剧情", "动作", "犯罪"],
            language="英语",
            rating=9.4
        ),
        "M008": Movie(
            movie_id="M008",
            title="复仇者联盟4",
            duration=181,
            genre=["动作", "科幻", "冒险"],
            language="英语",
            rating=8.5
        ),
        "M009": Movie(
            movie_id="M009",
            title="疯狂动物城",
            duration=108,
            genre=["动画", "冒险", "喜剧"],
            language="英语",
            rating=9.2
        ),
        "M010": Movie(
            movie_id="M010",
            title="哪吒之魔童降世",
            duration=110,
            genre=["动画", "奇幻", "冒险"],
            language="中文",
            rating=8.5
        )
    }
    
    # 将电影数据转换为可序列化的字典
    movies_dict = {movie_id: movie.model_dump() for movie_id, movie in movies.items()}
    
    # 写入JSON文件
    with open(MOVIES_FILE, 'w', encoding="utf-8") as f:
        json.dump(movies_dict, f, indent=4, ensure_ascii=False)
    
    return movies_dict

# 初始化影厅数据
def initialize_theaters_data():
    """初始化影厅数据并写入JSON文件"""
    theaters = {}
    
    for i in range(1, 6):
        theater_id = f"T00{i}"
        capacity = 100 if i <= 2 else (150 if i <= 4 else 200)
        theater_type = ["2D"] if i == 1 else (["3D"] if i == 2 else (["IMAX"] if i == 3 else (["2D", "3D"] if i == 4 else ["IMAX", "3D"])))
        
        seating_layout = {}
        rows = "ABCDEFGHIJ" if capacity <= 100 else "ABCDEFGHIJKL"
        seats_per_row = 10 if capacity <= 100 else (15 if capacity <= 150 else 20)
        
        for row in rows[:len(rows)]:
            for seat in range(1, seats_per_row + 1):
                seat_number = f"{row}{seat}"
                seating_layout[seat_number] = True
        
        theaters[theater_id] = Theater(
            theater_id=theater_id,
            name=f"影厅{i}",
            capacity=capacity,
            type=theater_type,
            seating_layout=seating_layout
        )
    
    # 将影厅数据转换为可序列化的字典
    theaters_dict = {theater_id: theater.model_dump() for theater_id, theater in theaters.items()}
    
    # 写入JSON文件
    with open(THEATERS_FILE, 'w', encoding="utf-8") as f:
        json.dump(theaters_dict, f, indent=4, ensure_ascii=False)
    
    return theaters_dict

# 初始化场次数据
def initialize_showtimes_data():
    """初始化场次数据并写入JSON文件"""
    showtimes = {}
    
    # 获取电影和影厅数据
    movies = load_movies_data()
    theaters = load_theaters_data()
    
    # 设置基准日期
    base_date = "2023-10-01"
    showtime_id_counter = 1
    
    # 为每个电影创建几个场次
    for movie_id, movie in movies.items():
        duration = movie["duration"]
        
        # 为每个影厅安排该电影的场次
        for theater_id, theater in theaters.items():
            # 只有电影类型和影厅类型匹配时才安排场次
            if "科幻" in movie["genre"] and "IMAX" in theater["type"]:
                # IMAX影厅优先安排科幻电影
                time_slots = [("09:00", "3D"), ("14:00", "IMAX"), ("19:00", "IMAX")]
            elif "动画" in movie["genre"] and "3D" in theater["type"]:
                # 3D影厅优先安排动画电影
                time_slots = [("10:00", "3D"), ("15:00", "3D")]
            else:
                # 其他情况随机安排
                time_slots = [("11:00", "2D"), ("16:00", "2D"), ("20:00", "2D")]
            
            # 根据电影时长计算结束时间并创建场次
            for time_slot in time_slots:
                start_time_str = f"{base_date} {time_slot[0]}"
                start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M")
                end_time = start_time + timedelta(minutes=duration)
                end_time_str = end_time.strftime("%Y-%m-%d %H:%M")
                
                # 仅当影厅支持该放映类型时才创建场次
                if time_slot[1] in theater["type"]:
                    showtime_id = f"S{showtime_id_counter:03d}"
                    showtime_id_counter += 1
                    
                    price = 40.0  # 基础票价
                    if "IMAX" in time_slot[1]:
                        price = 80.0  # IMAX票价
                    elif "3D" in time_slot[1]:
                        price = 60.0  # 3D票价
                    
                    showtimes[showtime_id] = Showtime(
                        showtime_id=showtime_id,
                        movie_id=movie_id,
                        theater_id=theater_id,
                        start_time=start_time_str,
                        end_time=end_time_str,
                        price=price,
                        available_seats=theater["capacity"],
                        booked_seats=[]
                    )
    
    # 将场次数据转换为可序列化的字典
    showtimes_dict = {showtime_id: showtime.model_dump() for showtime_id, showtime in showtimes.items()}
    
    # 写入JSON文件
    with open(SHOWTIMES_FILE, 'w', encoding="utf-8") as f:
        json.dump(showtimes_dict, f, indent=4, ensure_ascii=False)
    
    return showtimes_dict

# 初始化电影票文件
def initialize_tickets_file():
    """初始化电影票文件"""
    with open(TICKETS_FILE, 'w', encoding="utf-8") as f:
        json.dump({}, f, indent=4, ensure_ascii=False)

# 加载电影数据
def load_movies_data():
    """从JSON文件加载电影数据"""
    if not os.path.exists(MOVIES_FILE):
        return initialize_movies_data()
    
    with open(MOVIES_FILE, 'r', encoding="utf-8") as f:
        return json.load(f)

# 加载影厅数据
def load_theaters_data():
    """从JSON文件加载影厅数据"""
    if not os.path.exists(THEATERS_FILE):
        return initialize_theaters_data()
    
    with open(THEATERS_FILE, 'r', encoding="utf-8") as f:
        return json.load(f)

# 加载场次数据
def load_showtimes_data():
    """从JSON文件加载场次数据"""
    if not os.path.exists(SHOWTIMES_FILE):
        return initialize_showtimes_data()
    
    with open(SHOWTIMES_FILE, 'r', encoding="utf-8") as f:
        return json.load(f)

# 加载电影票数据
def load_tickets_data():
    """从JSON文件加载电影票数据"""
    if not os.path.exists(TICKETS_FILE):
        initialize_tickets_file()
        return {}
    
    with open(TICKETS_FILE, 'r', encoding="utf-8") as f:
        return json.load(f)

# 保存电影票数据
def save_ticket(ticket: Ticket):
    """保存电影票数据到JSON文件"""
    tickets = load_tickets_data()
    
    # 将新的电影票添加到字典中
    tickets[ticket.ticket_id] = ticket.model_dump()
    
    # 保存到文件
    with open(TICKETS_FILE, 'w', encoding="utf-8") as f:
        json.dump(tickets, f, indent=4, ensure_ascii=False)
    
    # 更新场次数据中的座位信息
    showtimes = load_showtimes_data()
    showtime = showtimes[ticket.showtime_id]
    showtime["booked_seats"].append(ticket.seat_number)
    showtime["available_seats"] -= 1
    
    with open(SHOWTIMES_FILE, 'w', encoding="utf-8") as f:
        json.dump(showtimes, f, indent=4, ensure_ascii=False)

# 更新影厅座位状态
def update_theater_seating(theater_id: str, seat_number: str, is_available: bool):
    """更新影厅座位状态"""
    theaters = load_theaters_data()
    
    if theater_id in theaters:
        theaters[theater_id]["seating_layout"][seat_number] = is_available
        
        with open(THEATERS_FILE, 'w', encoding="utf-8") as f:
            json.dump(theaters, f, indent=4, ensure_ascii=False)
        return True
    return False

# 获取所有电影信息
def get_all_movies() -> Dict:
    """获取所有电影信息"""
    return load_movies_data()

# 获取特定电影信息
def get_movie_info(movie_id: str) -> Dict:
    """获取特定电影信息"""
    movies = load_movies_data()
    return movies.get(movie_id, {"error": "电影不存在"})

# 获取特定电影的所有场次
def get_movie_showtimes(movie_id: str) -> List[Dict]:
    """获取特定电影的所有场次"""
    showtimes = load_showtimes_data()
    movie_showtimes = [showtime for showtime_id, showtime in showtimes.items() if showtime["movie_id"] == movie_id]
    return movie_showtimes

# 获取特定场次信息
def get_showtime_info(showtime_id: str) -> Dict:
    """获取特定场次信息"""
    showtimes = load_showtimes_data()
    return showtimes.get(showtime_id, {"error": "场次不存在"})

# 获取特定时间可观看的电影
def get_available_movies_by_time(start_time: str, end_time: str) -> List[Dict]:
    """获取特定时间可观看的电影"""
    showtimes = load_showtimes_data()
    movies = load_movies_data()
    
    start_dt = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
    end_dt = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
    
    available_showtimes = [
        showtime for showtime_id, showtime in showtimes.items()
        if datetime.strptime(showtime["start_time"], "%Y-%m-%d %H:%M") >= start_dt
        and datetime.strptime(showtime["end_time"], "%Y-%m-%d %H:%M") <= end_dt
        and showtime["available_seats"] > 0
    ]
    
    result = []
    for showtime in available_showtimes:
        movie_id = showtime["movie_id"]
        movie_info = movies.get(movie_id, {})
        result.append({
            "movie": movie_info,
            "showtime": showtime
        })
    
    return result

# 获取特定类型的电影
def get_movies_by_genre(genre: str) -> List[Dict]:
    """获取特定类型的电影"""
    movies = load_movies_data()
    return [movie for movie_id, movie in movies.items() if genre in movie["genre"]]

# 获取特定类型放映的场次
def get_showtimes_by_type(projection_type: str) -> List[Dict]:
    """获取特定类型放映的场次"""
    showtimes = load_showtimes_data()
    theaters = load_theaters_data()
    
    type_showtimes = []
    for showtime_id, showtime in showtimes.items():
        theater = theaters.get(showtime["theater_id"], {})
        if projection_type in theater.get("type", []):
            type_showtimes.append(showtime)
    
    return type_showtimes

# 预订电影票
def book_ticket(showtime_id: str, seat_number: str, user_name: str) -> Dict:
    """预订电影票"""
    showtimes = load_showtimes_data()
    
    # 检查场次是否存在
    if showtime_id not in showtimes:
        return {"error": "场次不存在"}
    
    showtime = showtimes[showtime_id]
    
    # 检查座位是否已被预订
    if seat_number in showtime["booked_seats"]:
        return {"error": "座位已被预订"}
    
    # 检查座位是否存在
    theaters = load_theaters_data()
    theater = theaters[showtime["theater_id"]]
    if seat_number not in theater["seating_layout"]:
        return {"error": "座位不存在"}
    
    # 创建电影票
    ticket_id = str(uuid4())
    ticket = Ticket(
        ticket_id=ticket_id,
        showtime_id=showtime_id,
        movie_id=showtime["movie_id"],
        theater_id=showtime["theater_id"],
        seat_number=seat_number,
        user_name=user_name,
        price=showtime["price"],
        booking_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    )
    
    # 保存电影票
    save_ticket(ticket)
    
    # 更新影厅座位状态
    update_theater_seating(showtime["theater_id"], seat_number, False)
    
    return {
        "success": True,
        "ticket_id": ticket_id,
        "message": f"成功预订电影票，票号: {ticket_id}"
    }

# 获取用户的预订历史
def get_user_bookings(user_name: str) -> List[Dict]:
    """获取用户的预订历史"""
    tickets = load_tickets_data()
    user_tickets = [ticket for ticket_id, ticket in tickets.items() if ticket["user_name"] == user_name]
    return user_tickets

# 查找可预订的相邻座位
def find_adjacent_seats(showtime_id: str, num_seats: int) -> List[str]:
    """查找可预订的相邻座位"""
    showtimes = load_showtimes_data()
    
    if showtime_id not in showtimes:
        return []
    
    showtime = showtimes[showtime_id]
    theaters = load_theaters_data()
    theater = theaters[showtime["theater_id"]]
    
    # 获取所有已预订的座位
    booked_seats = set(showtime["booked_seats"])
    
    # 按行查找连续的可用座位
    rows = "ABCDEFGHIJKL"
    available_adjacent_seats = []
    
    for row in rows:
        current_adjacent = []
        for seat_num in range(1, 21):  # 假设每行最多20个座位
            seat = f"{row}{seat_num}"
            if seat in theater["seating_layout"] and seat not in booked_seats:
                current_adjacent.append(seat)
                if len(current_adjacent) == num_seats:
                    available_adjacent_seats.append(current_adjacent.copy())
                    current_adjacent.pop(0)  # 滑动窗口继续寻找下一组
            else:
                current_adjacent = []  # 重置计数器
    
    return available_adjacent_seats

# 初始化程序
if __name__ == "__main__":
    initialize_movies_data()
    initialize_theaters_data()
    initialize_showtimes_data()
    initialize_tickets_file()

# 测试函数
def check(conditions):
    tickets = load_tickets_data()
    for ticket_id, ticket_data in tickets.items():
        results = [c(ticket_data) for c in conditions]
        r = all(results)
        if r:
            return 1
    return 0



In [ ]:
# 初始化数据防止干扰
initialize_movies_data()
initialize_theaters_data()
initialize_showtimes_data()
initialize_tickets_file()

# 测试题1：预订特定电影的票
def eval1():
    name = "Alice001"
    movie_title = "星际穿越"
    q1 = f"你在系统中的名字是{name}，请预订电影《{movie_title}》的任意一场次的票"
    conditions = [
        lambda t: t["user_name"] == name,
        lambda t: load_movies_data()[t["movie_id"]]["title"] == movie_title
    ]
    return q1, conditions

# 测试题2：预订特定类型的电影
def eval2():
    name = "Bob002"
    genre = "科幻"
    q2 = f"你在系统中的名字是{name}，请预订一部{genre}类型电影的票"
    conditions = [
        lambda t: t["user_name"] == name,
        lambda t: genre in load_movies_data()[t["movie_id"]]["genre"]
    ]
    return q2, conditions

# 测试题3：预订IMAX放映类型的电影
def eval3():
    name = "Charlie003"
    projection_type = "IMAX"
    q3 = f"你在系统中的名字是{name}，请预订一场{projection_type}类型放映的电影票"
    conditions = [
        lambda t: t["user_name"] == name,
        lambda t: projection_type in load_theaters_data()[t["theater_id"]]["type"]
    ]
    return q3, conditions

# 测试题4：预订最高评分的电影
def eval4():
    name = "David004"
    q4 = f"你在系统中的名字是{name}，请预订评分最高的电影的票"
    
    def is_highest_rated_movie(t):
        movies = load_movies_data()
        movie_id = t["movie_id"]
        movie_rating = movies[movie_id]["rating"]
        highest_rating = max(movie["rating"] for movie in movies.values())
        return movie_rating == highest_rating
    
    conditions = [
        lambda t: t["user_name"] == name,
        is_highest_rated_movie
    ]
    return q4, conditions

# 测试题5：预订特定时间段的电影
def eval5():
    name = "Eve005"
    start_time = "2023-10-01 14:00"
    end_time = "2023-10-01 16:00"
    q5 = f"你在系统中的名字是{name}，请预订{start_time}到{end_time}之间上映的任意电影票"
    
    def is_in_time_range(t):
        showtime = get_showtime_info(t["showtime_id"])
        showtime_start = datetime.strptime(showtime["start_time"], "%Y-%m-%d %H:%M")
        showtime_end = datetime.strptime(showtime["end_time"], "%Y-%m-%d %H:%M")
        start = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
        end = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        return showtime_start >= start and showtime_end <= end
    
    conditions = [
        lambda t: t["user_name"] == name,
        is_in_time_range
    ]
    return q5, conditions

# 测试题6：为多人预订相邻座位
def eval6():
    name = "Frank006"
    num_tickets = 3
    q6 = f"你在系统中的名字是{name}，请为{num_tickets}人预订同一场次的相邻座位"
    
    def has_adjacent_seats(t):
        tickets = load_tickets_data()
        user_tickets = [ticket for ticket_id, ticket in tickets.items() if ticket["user_name"] == name]
        if len(user_tickets) < num_tickets:
            return False
        
        # 检查是否为同一场次
        showtime_ids = {ticket["showtime_id"] for ticket in user_tickets}
        if len(showtime_ids) != 1:
            return False
        
        # 检查座位是否相邻
        seats = [ticket["seat_number"] for ticket in user_tickets]
        rows = {seat[0] for seat in seats}
        if len(rows) != 1:  # 确保所有座位在同一行
            return False
        
        # 提取座位号并排序
        seat_numbers = [int(seat[1:]) for seat in seats]
        seat_numbers.sort()
        
        # 检查座位号是否连续
        for i in range(len(seat_numbers) - 1):
            if seat_numbers[i + 1] - seat_numbers[i] != 1:
                return False
        
        return True
    
    conditions = [
        lambda t: t["user_name"] == name,
        has_adjacent_seats
    ]
    return q6, conditions

# 测试题7：预订特定座位
def eval7():
    name = "Grace007"
    seat = "C5"
    q7 = f"你在系统中的名字是{name}，请预订任意场次的{seat}座位"
    conditions = [
        lambda t: t["user_name"] == name,
        lambda t: t["seat_number"] == seat
    ]
    return q7, conditions

# 测试题8：预订特定语言的电影
def eval8():
    name = "Henry008"
    language = "日语"
    q8 = f"你在系统中的名字是{name}，请预订一部{language}电影的票"
    conditions = [
        lambda t: t["user_name"] == name,
        lambda t: load_movies_data()[t["movie_id"]]["language"] == language
    ]
    return q8, conditions

# 测试题9：预订时长最短的电影
def eval9():
    name = "Ivy009"
    q9 = f"你在系统中的名字是{name}，请预订时长最短的电影票"
    
    def is_shortest_movie(t):
        movies = load_movies_data()
        movie_id = t["movie_id"]
        movie_duration = movies[movie_id]["duration"]
        shortest_duration = min(movie["duration"] for movie in movies.values())
        return movie_duration == shortest_duration
    
    conditions = [
        lambda t: t["user_name"] == name,
        is_shortest_movie
    ]
    return q9, conditions

# 测试题10：预订不存在的场次
def eval10():
    name = "Jack010"
    showtime_id = "S999"
    q10 = f"你在系统中的名字是{name}，请尝试预订场次ID为{showtime_id}的电影票"
    conditions = [
        lambda t: t["user_name"] != name or t["showtime_id"] != showtime_id
    ]
    return q10, conditions

In [2]:
# 获取星际穿越的movie_id
movies = get_all_movies()
interstellar_id = "M001"  # 星际穿越的ID

# 获取该电影的场次
showtimes = get_movie_showtimes(interstellar_id)
available_showtime = showtimes[0]["showtime_id"]  # 选择第一个场次

# 预订票
result = book_ticket(available_showtime, "A1", "Alice001")
print("测试1结果:", check(eval1()[1]))  # 应为1（成功）

测试1结果: 1


In [3]:
# 获取科幻类型的电影
sci_fi_movies = get_movies_by_genre("科幻")
sci_fi_movie_id = sci_fi_movies[0]["movie_id"]  # 选择第一个科幻电影

# 获取该电影的场次
showtimes = get_movie_showtimes(sci_fi_movie_id)
available_showtime = showtimes[0]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "B2", "Bob002")
print("测试2结果:", check(eval2()[1]))  # 应为1（成功）

测试2结果: 1


In [4]:
# 获取IMAX类型放映的场次
imax_showtimes = get_showtimes_by_type("IMAX")
available_showtime = imax_showtimes[0]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "C3", "Charlie003")
print("测试3结果:", check(eval3()[1]))  # 应为1（成功）

测试3结果: 1


In [5]:
# 获取所有电影
movies = get_all_movies()
# 找出评分最高的电影
highest_rated_movie = max(movies.values(), key=lambda x: x["rating"])
highest_rated_movie_id = highest_rated_movie["movie_id"]

# 获取该电影的场次
showtimes = get_movie_showtimes(highest_rated_movie_id)
available_showtime = showtimes[0]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "D4", "David004")
print("测试4结果:", check(eval4()[1]))  # 应为1（成功）

测试4结果: 1


In [6]:
# 获取14:00-16:00之间上映的电影
available_movies = get_available_movies_by_time("2023-10-01 14:00", "2023-10-01 16:00")
available_showtime = available_movies[0]["showtime"]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "E5", "Eve005")
print("测试5结果:", check(eval5()[1]))  # 应为1（成功）

测试5结果: 1


In [7]:
# 找到一个有足够相邻座位的场次
showtimes = load_showtimes_data()
showtime_id = list(showtimes.keys())[0]  # 选择第一个场次
adjacent_seats = find_adjacent_seats(showtime_id, 3)[0]  # 获取3个相邻座位

# 预订3个相邻座位
for seat in adjacent_seats:
    result = book_ticket(showtime_id, seat, "Frank006")
print("测试6结果:", check(eval6()[1]))  # 应为1（成功）

测试6结果: 1


In [8]:
# 找到一个C5座位未被预订的场次
showtimes = load_showtimes_data()
for showtime_id, showtime in showtimes.items():
    if "C5" not in showtime["booked_seats"]:
        # 预订票
        result = book_ticket(showtime_id, "C5", "Grace007")
        break
print("测试7结果:", check(eval7()[1]))  # 应为1（成功）

测试7结果: 1


In [9]:
# 获取所有电影
movies = get_all_movies()
# 找出日语电影
japanese_movies = [movie for movie_id, movie in movies.items() if movie["language"] == "日语"]
japanese_movie_id = japanese_movies[0]["movie_id"]  # 千与千寻

# 获取该电影的场次
showtimes = get_movie_showtimes(japanese_movie_id)
available_showtime = showtimes[0]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "H8", "Henry008")
print("测试8结果:", check(eval8()[1]))  # 应为1（成功）

测试8结果: 1


In [10]:
# 获取所有电影
movies = get_all_movies()
# 找出时长最短的电影
shortest_movie = min(movies.values(), key=lambda x: x["duration"])
shortest_movie_id = shortest_movie["movie_id"]  # 楚门的世界，103分钟

# 获取该电影的场次
showtimes = get_movie_showtimes(shortest_movie_id)
available_showtime = showtimes[0]["showtime_id"]

# 预订票
result = book_ticket(available_showtime, "I9", "Ivy009")
print("测试9结果:", check(eval9()[1]))  # 应为1（成功）

测试9结果: 1


In [11]:
# 尝试预订不存在的场次
result = book_ticket("S999", "J10", "Jack010")
print("测试10结果:", check(eval10()[1]))  # 应为1（因为条件检查预订是否失败）

测试10结果: 1
